In [1]:
import pandas as pd
import numpy as np
import itertools
import joblib

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from random import randint

In [2]:
start_data = pd.read_excel('data_with_point.xlsx')
mas_of_letters = []
for row in start_data.iterrows():
    if row[1][8] == 'Т':
        mas_of_letters.append(1)
    elif row[1][8] == 'С':
        mas_of_letters.append(2)
    else:
        mas_of_letters.append(3)
start_data['J'] = mas_of_letters
start_data.head()

,A,B,C,D,E,F,G,H,J,Points
0,55,16,84,0.98,3,6,91,8,2,68.846154
1,81,5,71,0.81,2,1,98,43,2,89.615385
2,99,28,48,0.81,3,0,41,12,2,73.461538
3,61,13,23,0.70,2,1,84,87,2,68.846154
4,95,17,81,0.10,5,3,83,21,2,72.307692


In [3]:
target = start_data['Points']
features = start_data.drop('Points', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=9)


In [6]:
data_with_predict = pd.read_excel('data_with_predict.xlsx')
data_predict_stat = pd.read_excel('data_predict_stat.xlsx')
mas_stat = []
count = 15
#adam_tanh_12
for el in ['adam']:
    for i in ['tanh']:
        try:
            params = {'activation': [i],
                      'hidden_layer_sizes': list(itertools.permutations([randint(1,600),randint(1,600),randint(1,600)],2)) + list(itertools.permutations([randint(1,300),randint(1,300),randint(1,300)],3)) + [randint(1,300),randint(1,200)],
                      'solver': [el],
                      'learning_rate' : ['adaptive']
                     }
            mlp_regressor_grid = GridSearchCV(MLPRegressor(random_state=123), param_grid=params, n_jobs=-1, cv=5, verbose=5)
            mlp_regressor_grid.fit(X_train,y_train)
            print(f'{el} {i} {count}')

    #         print(X_test)
    #         print(y_test)
            train_score = mlp_regressor_grid.best_estimator_.score(X_train, y_train)
            test_score = mlp_regressor_grid.best_estimator_.score(X_test, y_test)
            best_score = mlp_regressor_grid.best_score_
            best_params = mlp_regressor_grid.best_params_

            print('Train R^2 Score : %.3f'%train_score)
            print('Test R^2 Score : %.3f'%test_score)
            print('Best R^2 Score Through Grid Search : %.3f'%best_score)
            print('Best Parameters : ',best_params)
            data_predict_stat[f'{el}_{i}_{count}'] = [train_score, test_score, best_score, best_params]
            best_model = mlp_regressor_grid.best_estimator_
            data_with_predict[f'{el}_{i}_{count}'] = best_model.predict(X_test)
            joblib.dump(best_model, f'{el}_{i}_{count}.joblib')
            count += 1
        except Exception  as e:
            print(e)


writer_predict = pd.ExcelWriter('data_with_predict.xlsx', engine='xlsxwriter')
data_with_predict.to_excel(writer_predict,'sheet1')
writer_predict.save()

writer_stat = pd.ExcelWriter('data_predict_stat.xlsx', engine='xlsxwriter')
data_predict_stat.to_excel(writer_stat,'sheet1')
writer_stat.save()

Fitting 5 folds for each of 14 candidates, totalling 70 fits


C:\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


adam tanh 15
Train R^2 Score : 0.866
Test R^2 Score : 0.813
Best R^2 Score Through Grid Search : 0.770
Best Parameters :  {'activation': 'tanh', 'hidden_layer_sizes': (229, 276), 'learning_rate': 'adaptive', 'solver': 'adam'}
